# This notebook can be used to process the dashcam data
## Data Process
1. Convert videos to frames. Store in `data/images/frames/`
2. Iterate over data in two locations: `data/images/frames` and `data/images/augmented` with 3 options:
    1. Label current frame:
        * Copies original image as a `768 X 448`
        * Draw bounding boxes
        * Save two formats of labels: `labels/viewing/` and `labels/formatted/`
        * Move `768 X 448` image to `data/images/processed/`
        * Move original image to `data/images/frames_original`
    2. Select frame for augmenting:
        * Move original image to `data/images/for_augmenting`
    3. Delete frame from dataset



In [10]:
import os
import sys
import cv2
sys.path.append(os.path.abspath('../src'))
import utils
import shutil

In [11]:
# Directories
trimmed_video_dir = '../data/videos/trimmed'

processed_video_dir = '../data/videos/processed'

frame_dir = '../data/images/frames'



In [12]:
videos = os.listdir(trimmed_video_dir)

num_videos = int(input(f"Enter the number of videos to convert to frames 0-{len(videos)}"))

num_videos = min(num_videos, len(videos))

print("\n".join(videos[:num_videos]))



20250222_152040M.mp4


In [13]:
for video in videos[:num_videos]:
    
    video_name = os.path.splitext(os.path.basename(video))[0]

    video_path = f"{trimmed_video_dir}/{video}"

    utils.video_to_frames(video_path, frame_dir, video_name)

    shutil.move(video_path, f"{processed_video_dir}/{video}")


Extracted 4264 frames at 30.0 FPS.


In [ ]:
# Standardized image width/height
IMAGE_WIDTH, IMAGE_HEIGHT = 768, 448

# Directory to place images once processed (resized and annotated)
processed_dir = "../data/images/processed"

# Directory to place original images
original_dir = "../data/images/frames_original"

# Directory to place images for augmenting
augmenting_dir = "../data/images/for_augmenting"

# Color mapping based on key presses
color_mapping = {
    ord("1"): (0, 255, 0),   # Green
    ord("2"): (0, 0, 255),   # Red
    ord("3"): (0, 255, 255), # Yellow
}

# Label mapping based on colors
label_mapping = {
    (0, 255, 0): "Green Light",
    (0, 0, 255): "Red Light",
    (0, 255, 255): "Yellow Light"
}

# Class mapping based on label (for YOLO format that uses int instead of str)
class_mapping = {
    "Green Light": 1,
    "Red Light": 2,
    "Yellow Light": 3
}

# Exit labelling variable
exit = False

# Mouse callback function
def draw_rectangle_with_drag(event, x, y, flags, param):
    global ix, iy, drawing, img_copy, img, current_color

    # Left clicking starts a drawing event if the user is not currently drawing
    if event == cv2.EVENT_LBUTTONDOWN and not drawing:  
        drawing = True # event status is drawing
        ix, iy = x, y # anchor point for the first corner of the rectangle
        img_copy = img.copy()  # reset copy when starting a new rectangle

    # When the cursor is moving and we are in drawing status display adjusted size of rectangle based on cursor location
    elif event == cv2.EVENT_MOUSEMOVE and drawing:  
        img_copy = img.copy()  # reset to avoid multiple overlapping rectangles
        cv2.rectangle(img_copy, (ix, iy), (x, y), current_color, 1) # drawing rectangle from ix, iy to current cursor position

    # Left click when we are already drawing places the rectangle where the cursor is located during the click
    elif event == cv2.EVENT_LBUTTONDOWN and drawing:  
        drawing = False # reset event status to not drawing
        cv2.rectangle(img, (ix, iy), (x, y), current_color, 1)  # draw on final image
        annotations.append({
            "x1": min(ix, x),
            "x2": max(ix, x),
            "y1": min(iy, y),
            "y2": max(iy, y),
            "color_code": current_color,
            "color": label_mapping[current_color],
            "class": class_mapping[label_mapping[current_color]]
        }) # appends a map of values needed for documentation min/max x and y coordinates, color codes, colors, and class
        
        redraw_rectangles()  # for view consistency

# redraws all the rectangles from the list of annotations
def redraw_rectangles():
    global img, img_copy
    img = resized_img.copy()  # reset to the resized image
    for ann in annotations:
        cv2.rectangle(img, 
                      (ann["x1"], ann["y1"]), 
                      (ann["x2"], ann["y2"]), 
                      ann["color_code"], 1) # draws each rectangle
        
    img_copy = img.copy()  # Update the display copy

# Iterate over each file in the frame dir
for filename in os.listdir(frame_dir):

    if exit:
        break

    file_path = os.path.join(frame_dir, filename)

    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Load image
        original_img = cv2.imread(file_path)  # keep original image
        if original_img is None:
            raise FileNotFoundError("Image not found. Check the file path.")
        
        # Resize the image to a uniform size
        resized_img = cv2.resize(original_img, (IMAGE_WIDTH, IMAGE_HEIGHT))

        # Initialize working images
        img = resized_img.copy()   # Active drawing image
        img_copy = img.copy()      # Image copy for real-time updates

        # Anchor variables
        ix, iy = -1, -1
        drawing = False
        current_color = (0, 255, 0)  # Default: Green

        # Prepare annotation file
        img_filename = os.path.basename(file_path)
        text_filename = os.path.splitext(img_filename)[0] + ".txt"
        
        # Viewing annotation file
        viewing_annotation_path = f"../data/labels/viewing/viewing_{text_filename}"
        # Formatted annotation file
        yolo_annotations_path = f'../data/labels/formatted/{text_filename}'


        # List to store dicts of annotations
        annotations = []
        # Create window and set mouse callback
        cv2.namedWindow(f"Label Data: {filename}", cv2.WINDOW_NORMAL)  # Allow resizing
        cv2.resizeWindow(f"Label Data: {filename}", IMAGE_WIDTH, IMAGE_HEIGHT)  # Set initial size
        cv2.setWindowProperty(f"Label Data: {filename}", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        cv2.setMouseCallback(f"Label Data: {filename}", draw_rectangle_with_drag)

        # Display loop
        while True:
            cv2.imshow(f"Label Data: {filename}", img_copy)  # Show dynamic updates
            key = cv2.waitKey(10) & 0xFF
            
            # Press 'Esc' to exit
            if key == 27:
                exit = True
                break
            
            # Press 's' to save annotations
            elif key == ord("s"):

                # Open viewing text file, iterate over annotations and write to file
                with open(viewing_annotation_path, 'w') as viewing_file:
                    for annotation in annotations:
                        viewing_file.write(f"{annotation}\n")
                print(f"Viewing annotations saved to {viewing_annotation_path}")
                
                # Convert annotations to yolo format
                yolo_annotations = utils.viewing_to_yolo(annotations, IMAGE_WIDTH, IMAGE_HEIGHT)

                # Open yolo text file, iterate over annotations and write to file
                with open(yolo_annotations_path, 'w') as yolo_file:
                    for yolo_ann in yolo_annotations:
                        yolo_str = ", ".join(map(str, yolo_ann))  # Convert each item to string and join with commas
                        yolo_file.write(f"{yolo_str}\n")  # Write formatted string to file
                print(f"YOLO annotations saved to {yolo_annotations_path}")


                # Open processed_dir and write resized image to the directory
                os.makedirs(processed_dir, exist_ok=True)
                processed_path = os.path.join(processed_dir, os.path.basename(file_path))
                cv2.imwrite(processed_path, resized_img)
                print(f"Moved {filename} -> {processed_path}")

                os.makedirs(original_dir, exist_ok=True)
                original_path = os.path.join(original_dir, os.path.basename(file_path))
                shutil.move(file_path, original_path)
                break


            # Press 'a' to move frame to data/images/for_augmenting
            elif key == ord('a'):
                for_augmenting_path = os.path.join(augmenting_dir, os.path.basename(file_path))
                shutil.move(file_path, for_augmenting_path)
                print(f"Moved {filename} -> {for_augmenting_path}")
                
            # Change rectangle color based on number key
            elif key in color_mapping:  
                current_color = color_mapping[key]
                print(f"Class changed to: {label_mapping[color_mapping[key]]}")
            
            # Press 'z' to undo last rectangle
            elif key == ord("z") and annotations: 
                annotations.pop()  # remove last rectangle
                redraw_rectangles()  # reset image and redraw remaining rectangles
                print("Last rectangle removed!")

        cv2.destroyAllWindows()
cv2.destroyAllWindows()